# **03. Reweighting a candidate ensemble of structures to fit the experimental HDX-MS data**

In the previous notebook, we calculated the predicted HDX-MS data for a candidate ensemble of BPTI structures taken from unbiased MD trajectories. We discovered that, for some backbone amides, the `BestVendruscolo` predictive model provided a good estimate of the experimental HDX-MS data, but substantial differences remained between theory and experiment for many residues. 

In the next few notebooks, we'll reduce those discrepancies by applying maximum entropy ensemble reweighting, and evaluate the effects of reweighting upon the relative populations of BPTI conformations present in the candidate ensemble. Ultimately, we weill identify a final, reweighted, conformational ensemble that best conforms to the experimental HDX-MS data, taking into account all sources of uncertainty in the fitting of theory and experiment.

First, we will create and run a short Python script, making use of the `HDXer.reweighting` module of the HDXer package, to apply a maximum-entropy-based bias to the same ensemble of structures that we analyzed using **calc_hdx.py** in the previous notebook.  More details of the teoroy behind the approach can be found in the publication below:

[Bradshaw, R. T., Marinelli, F. et al. (2020) 'Interpretation of HDX Data by Maximum-Entropy Reweighting of Simulated Structural Ensembles', Biophysical Journal, 118(7), 1649-1664](https://www.cell.com/biophysj/fulltext/S0006-3495(20)30124-7)

### **Necessary inputs**

We have already generated or provided all of the input files necessary to carry out ensemble reweighting with HDXer. When we create a Python script below to actually carry out the reweighting, we'll have to provide the following arguments to the `HDXer.reweighting.MaxEnt` object in order to define our reweighting analysis options:
- data_folders: directory/ies with the output `*.tmp` files of per-residue contacts & H-bonds, created by **calc_hdx.py**
- exp_file: file containing the target (experimental) HDX-MS data
- kint_file: file containing the per-residue intrinsic rates. You can use the one created by **calc_hdx.py**
- times: HDX-MS labeling times, in minutes, for the target (experimental) data

There is also one key parameter that we'll have to choose for each reweighting run, but that we haven't yet discussed:
- gamma: $\gamma$ value for the current reweighting analysis

### **$\gamma$ values in ensemble reweighting**

Conceptually, the value of $\gamma$ is related to the level of uncertainty included in the reweighting of the candidate ensemble to fit the target data. Equation 7 of [Bradshaw, Marinelli et al.](https://www.cell.com/biophysj/fulltext/S0006-3495(20)30124-7) contains details of the quantitative meaning of $\gamma$, but we'll also provide a qualitative explanation of why its a crucial parameter for reweighting below.

The magnitude of $\gamma$ and the level of uncertainty included within a reweighting analysis are inversely related. Conceptually then, as the $\gamma$ value is increased, a greater bias will be applied to the underlying ensemble, the reweighting will fit more tightly to the target data, and the error between the final predicted HDX-MS and the target experimental HDX-MS data will be reduced.

However, reliable data fitting requires robustness checks. Overfitting, in which the predicted and target data agree more precisely than the true level of uncertainty in the target data, can readily occur. The 'true' uncertainty in our reweighting arises from multiple different sources, and hence we don't know the 'correct' value of $\gamma$ to use *a priori*. We therefore perform multiple reweighting analyses, each with a unique $\gamma$ value, and select the optimal $\gamma$ value using some common heuristics. This will generate a robust final reweighted structural ensemble for further analyses.

In this tutorial, we'll explore a range of $\gamma$ values from $1*10^{-3}$ to $9*10^{0}$. In our experience this range is typically suitable for applications of HDXer to reweight biomolecular MD simulations, but when first designing experiments for a new system we recommend exploring as wide a range of $\gamma$ values as possible, to ensure you've fully characterized the level of uncertainty present in your own analyses.

### **Example reweighting Python script**

*N.B. In this section, we'll set up and run an series of reweighting analyses of our BPTI data. These should take approximately 15-20 mins to complete. If you don't have time to complete the analyses in full, we have provided example output files within the tutorials directly, so you can skip the Python code cells below, and move to the analysis of the provided data.*

To run reweighting, we'll create a `HDXer.reweighting.MaxEnt` object that contains all the options for our analysis. Then to actually perform the iterative reweighting process, we'll invoke the `MaxEnt.run` method.

Here is an example set of commands to run HDX reweighting using the contacts, H-bonds, and per-residue intrinsic rates calculated in notebook 2 by **calc_hdx.py**, the experimental HDX-MS data for BPTI created in notebook 1, and a $\gamma$ value of $10^{-3}$. The cell below should take up to 1-2 mins to complete:

In [ ]:
#!/usr/bin/env python

import os

# Import the Maximum Entropy reweighting class
from HDXer.reweighting import MaxEnt

### Inputs ###

# A list of folders that contain the 'Contacts_' and 'Hbonds_' files from calc_hdx
folders = [ os.path.expandvars("$HDXER_PATH/tutorials/BPTI/BPTI_calc_hdx") ]
# The path to the target experimental data file
expt = os.path.expandvars("$HDXER_PATH/tutorials/BPTI/BPTI_expt_data/BPTI_expt_dfracs.dat")
# The path to the file containing intrinsic rates for each residue in your protein, generated by calc_hdx
rates = os.path.expandvars("$HDXER_PATH/tutorials/BPTI/BPTI_calc_hdx/BPTI_Intrinsic_rates.dat")
# A list of timepoints in the experimental data (in minutes)
times = [ 0.167, 1.0, 10.0, 120.0 ]


### Running reweighting ###

# These lines will run reweighting for a gamma value of 1 x 10^-3
# Adapt them as necessary
exponent = -3
basegamma = 10**exponent

reweight_object = MaxEnt(do_reweight=True, do_params=False, stepfactor=0.00001)
reweight_object.run(gamma=basegamma, data_folders=folders, kint_file=rates, exp_file=expt, times=times, restart_interval=100, out_prefix=f'reweighting_gamma_1x10^{exponent}_')
print(f'Reweighting for gamma = 1x10^{exponent} completed')

# Help text describing options and how to call the reweighting functions
# is available in the docstrings of the MaxEnt class, e.g.:
#help(MaxEnt)
#help(MaxEnt.run)

The commands used to actually run the reweighting analysis are split into two parts:
1. We create a `MaxEnt` object with three options that define the type of analysis we want to perform. In this case, we will switch *on* a maximum entropy reweighting (`do_reweight=True`), switch *off* optimization of the $\beta_{C}$ and $\beta_{H}$ parameters used to calculate the residue protection factors at every reweighting step (`do_params=False`), and set a very small stepsize for the iterative reweighting process (`stepfactor=0.00001`).
2. We run reweighting by calling the `MaxEnt.run` method, with arguments to define the necessary inputs of the location of our residue-level contacts and H-bonds data, the intrinsic rates, the target data, and the target data labeling times. We also provide two optional arguments to denote that we wish to write a restart file every 100 reweighting steps (`restart_interval`), and a prefix for all of our output files (`out_prefix`).



A number of other options are available, and we suggest you make use of the in-built help commands (`help(MaxEnt)` or `help(MaxEnt.run)`) to explore them fully. For example, if you wish to optimize the $\beta_{C}$ and $\beta_{H}$ parameters you can do so using a gradient-descent method (the default), using a Monte-Carlo minimization, or using Monte-Carlo sampling. The keywords required for each approach are detailed in the `MaxEnt` help text!

For now, though, we'll complete our reweighting analysis by repeating the same commands across a range of different $\gamma$ values. This is as simple as writing a loop! The following cell should take up to 10-15 minutes to complete:

In [ ]:
# This loop will run reweighting for gamma values of 2 x 10^-3 to 9 x 10^-3
# Adapt it as necessary
for multiplier in range(2, 10):
    reweight_object = MaxEnt(do_reweight=True, do_params=False, stepfactor=0.00001)
    reweight_object.run(gamma=(multiplier * basegamma), data_folders=folders, kint_file=rates, exp_file=expt, times=times, restart_interval=100, out_prefix=f'reweighting_gamma_{multiplier}x10^{exponent}_')
    print(f'Reweighting for gamma = {multiplier}x10^{exponent} completed')

Having completed the above scripts, we'll have performed reweighting across a range of $\gamma$ values from $1*10^{-3}$ to $9*10^{-3}$. However, for BPTI, these small $\gamma$ values reflect a large amount of uncertainty included in the reweighting analysis, which means very little bias will have been applied to modify the relative conformational populations in the candidate ensemble, which means the reweighted ensembles will show very little improvement between the predicted and experimental HDX-MS data.

Larger $\gamma$ values must be explored to apply a greater level of bias to the underlying ensemble. If you wish, you can perform this analysis yourself, and carry out reweighting with $\gamma$ values up to $\gamma = 10^1$, say. However, as more bias is applied, each subsequent reweighting analysis will take a greater number of iterations to converge, and it is likely you'll have to increase the iteration step size (`stepfactor`) to efficiently converge reweighting analyses at higher $\gamma$ values

To save time therefore, for the remaining notebooks, we provide example data that can be used to analyze the results of reweighting.

The output files from running reweighting detail:
1. The initial parameters used for the reweighting, for record keeping
2. The final predicted deuterated fractions after reweighting
3. The difference (mean square error and root mean square error) between the predicted and target HDX-MS data, and the 'apparent work' applied as a bias to the ensemble as a whole
4. The individual relative weights (i.e. probabilities) of each frame in the structural ensemble. Note that as a _reweighting_ protocol, these weights will never be reduced to zero. All frames present in the initial structural ensemble will be present in the final ensemble, with varying, positive, and finite relative weights
5. Intermediate progress files showing the convergence of key reweighting metrics over each iteration (useful for debugging), or each restart file (useful for tracking progress)
6. A restart `*.pkl` file that can be used to restart a reweighting analysis that has been interrupted.

These output files will be used in the following notebooks to analyze our reweighting results, and visualize the changes that reweighting has made to our predicted HDX-MS data and to the conformational populations in our BPTI structural ensemble. We'll start by choosing an optimal $\gamma$ value at which to interrogate the results, in the next notebook *04_decision_plot.ipynb*.